In [2]:
import os
import pandas as pd
import json

In [36]:
def get_src_text(row):
    path = "data/en/" + row['event'] + "/" + str(row['threadid']) + "/source-tweets/" + str(row['tweetid']) + ".json"
    with open(path, "r") as f:
        source = json.loads(f.read())
        return source['text']

def is_true(row):
    print(row)
    path = "data/en/" + row['event'] + "/" + str(row['threadid']) + "/annotation.json"
    with open(path, "r") as f:
        source = json.loads(f.read())
        return str(source.get('true', 'unverified'))

In [41]:
# Dataset used: https://figshare.com/articles/dataset/PHEME_rumour_scheme_dataset_journalism_use_case/2068650
df = pd.read_json("data/en-scheme-annotations.json", dtype = {"threadid": str, "tweetid": str}, lines=True)

df['true'] = df.apply(lambda row: is_true(row), axis=1)
df['src_text'] = df.apply(lambda row: get_src_text(row), axis=1)

event                  putinmissing
threadid         577258317942149120
tweetid          577258317942149120
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 0, dtype: object
event                  putinmissing
threadid         576755174531862529
tweetid          576755174531862529
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 1, dtype: object
event                  putinmissing
threadid         576319832800555008
tweetid          576319832800555008
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 2, dtype: object
event                  putinmissing
threadid         576513463738109954
tweetid          576513463738109954
support                     denying
evidentiality             url-given
certainty                   certain
Name: 3, dtype: object
event                  charliehebdo
threadid

event             germanwings-crash
threadid         580339547269144576
tweetid          580339547269144576
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 133, dtype: object
event                ottawashooting
threadid         525003468659228672
tweetid          525003468659228672
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 134, dtype: object
event                ottawashooting
threadid         524975705206304769
tweetid          524975705206304769
support                  supporting
evidentiality           no-evidence
certainty                   certain
Name: 135, dtype: object
event                ottawashooting
threadid         524942470472548352
tweetid          524942470472548352
support                  supporting
evidentiality             witnessed
certainty                   certain
Name: 136, dtype: object
event                ottawashooting


event                ottawashooting
threadid         524959809402331137
tweetid          524959809402331137
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 274, dtype: object
event                ottawashooting
threadid         525049639016615937
tweetid          525049639016615937
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 275, dtype: object
event                ottawashooting
threadid         524966904885428226
tweetid          524966904885428226
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 276, dtype: object
event                ottawashooting
threadid         524995771587108864
tweetid          524995771587108864
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 277, dtype: object
event                ottawashooting


2